<a href="https://colab.research.google.com/github/ilonayoyoo/100-Days-of-Code-Data-Science/blob/main/5400HW5-yw4113.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")
crunchbase = "/content/drive/MyDrive/crunchbase_odm_orgs.csv"

Mounted at /content/drive


In [3]:
!pip install spark

!pip install pyspark



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for spark: filename=spark-0.2.1-py3-none-any.whl size=58749 sha256=74e0dd42cfb9dc5a990008729dc77b39ca81416a5d69435d544c5e075207f1df
  Stored in directory: /root/.cache/pip/wheels/63/88/77/b4131110ea4094540f7b47c6d62a649807d7e94800da5eab0b
Successfully built spark
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=d0ab4219271a255dcdea36602e421efb27e193508dd277f6be7d7e1e45a7f4a5
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [4]:
import json
import csv
import spark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_extract, when, reverse, udf
from pyspark.sql.types import BooleanType

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
# Gets or creates a new Spark session
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
print("Using Apache Spark Version", spark.version)

Using Apache Spark Version 3.5.1


In [6]:
import os, sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [10]:
spark = SparkSession.builder \
    .appName("crunchbase") \
    .getOrCreate()

df = spark.read.csv(crunchbase, header=True)

Q1: Find all companies with the name that is only two words (e.g. : "Goldman Sachs") print the count of such companies and show()only the name and location (city, region, country_code) in the resulting Spark DataFrame

In [16]:
from pyspark.sql.functions import split
from pyspark.sql.functions import size


df1 = df.withColumn("words", split(df.name, " "))
df1 = df1.withColumn("word_count", size(df1.words))
df1_filter = df1.filter(df1["word_count"] == 2)

In [17]:
df1_filter.count()

362534

In [18]:
df1_filter.show()

+--------------------+--------------------+------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------------+----------------+----------+------------+--------------------+--------------------+----------+
|                uuid|                name|        type|primary_role|              cb_url|              domain|        homepage_url|            logo_url|        facebook_url|         twitter_url|        linkedin_url|combined_stock_symbols|            city|    region|country_code|   short_description|               words|word_count|
+--------------------+--------------------+------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------------+----------------+----------+------------+--------------------+--------------------+----------

In [12]:
df.filter(regexp_extract(col('name'), r'^\w+\s+\w+$', 0) != '')[['name', 'city', 'region', 'country_code']].show()

+--------------------+----------------+----------+------------+
|                name|            city|    region|country_code|
+--------------------+----------------+----------+------------+
|         Time Warner|        New York|  New York|         USA|
|       Goldman Sachs|        New York|  New York|         USA|
|     Jingle Networks|        New York|  New York|         USA|
|Hearst Communicat...|        New York|  New York|         USA|
|    Ning Interactive|      Menlo Park|California|         USA|
| Prosper Marketplace|   San Francisco|California|         USA|
|       Tribune Media|         Chicago|  Illinois|         USA|
| Aggregate Knowledge|       San Mateo|California|         USA|
|        Zing Systems|   Mountain View|California|         USA|
|         Amie Street|Long Island City|  New York|         USA|
|          Legg Mason|       Baltimore|  Maryland|         USA|
|        Haute Secure|         Seattle|Washington|         USA|
|          Squid Labs|         Alameda|C

In [13]:
df.filter(regexp_extract(col('name'), r'^\w+\s+\w+$', 0) != '').count()

336766

Q2: Find all companies located in the state of California: print the count of such companies and show()only the name and location (city, region, country_code) in the resulting Spark DataFrame

In [19]:
df[df['region'] == 'California'][['name', 'city', 'region', 'country_code']].show()


+--------------------+--------------+----------+------------+
|                name|          city|    region|country_code|
+--------------------+--------------+----------+------------+
|                Zoho|    Pleasanton|California|         USA|
|            Facebook|    Menlo Park|California|         USA|
|               Accel|     Palo Alto|California|         USA|
|           Omnidrive|     Palo Alto|California|         USA|
|                Geni|West Hollywood|California|         USA|
|             Flektor|   Culver City|California|         USA|
|Fox Interactive M...| Beverly Hills|California|         USA|
|             Twitter| San Francisco|California|         USA|
|         StumbleUpon| San Francisco|California|         USA|
|              Scribd| San Francisco|California|         USA|
|             Slacker|     San Diego|California|         USA|
|                Lala|     Palo Alto|California|         USA|
|               Helio|   Los Angeles|California|         USA|
|       

In [20]:
df[df['region'] == 'California'].count()

94871

**Question 3: Add a "Blog" column to the DataFrame with the row entries set to 1 if the "domain" field contains "blogspot.com", and 0 otherwise.** <br> *show() only the name, location (city, region, country_code) and "Blog" column for companies with the "Blog" field marked as 1*

In [24]:
df = df.withColumn("Blog", when(df['domain'].like('%blogspot.com%'), 1).otherwise(0))
df[df["Blog"] == 1][['name', 'city', 'region', 'country_code', 'Blog']].show()

+--------------------+-------------+------------+------------+----+
|                name|         city|      region|country_code|Blog|
+--------------------+-------------+------------+------------+----+
|     Sad Urdu Poetry|  San Antonio|       Texas|         USA|   1|
|      The Tech-Freak|    Sheffield|   Sheffield|         GBR|   1|
|           Ma.Gnolia|San Francisco|  California|         USA|   1|
|      Dynasty Online|         NULL|        NULL|        NULL|   1|
|            Hire-seo|         NULL|        NULL|        NULL|   1|
|          YelloYello|     Rijswijk|Zuid-Holland|         NLD|   1|
|       Youtubehiphop|    São Paulo|   Sao Paulo|         BRA|   1|
|     Payday advances|         NULL|        NULL|        NULL|   1|
|Blog Traffic Exch...|   Menlo Park|  California|         USA|   1|
|Sirius Forex Trad...|         NULL|        NULL|        NULL|   1|
|          Utilsforge|     Delaware|        Ohio|         USA|   1|
|      Discover India|    Faridabad|     Haryana

In [25]:
df[df["Blog"] == 1].count()

394


Question 4: Find all companies with names that are palindromes (name reads the same way forward and reverse, e.g. madam) using Spark UDF function

In [26]:
from pyspark.sql.types import BooleanType
from pyspark.sql.functions import udf

def is_palindrome(name):
    if name is None:
        return False
    name = name.lower()
    return name == name[::-1]

palindrome_company = udf(is_palindrome, BooleanType())

In [27]:
df4 = df.filter(palindrome_company(df["name"]))

In [28]:
df4.select("name").count()

1132

In [30]:
df4[['name', 'city', 'region', 'country_code', 'Blog']].show()

+-------+--------------------+--------------------+------------+----+
|   name|                city|              region|country_code|Blog|
+-------+--------------------+--------------------+------------+----+
| Revver|         Los Angeles|          California|         USA|   0|
|  KAYAK|            Stamford|         Connecticut|         USA|   0|
|  ooVoo|            New York|            New York|         USA|   0|
|Imikimi|             Seattle|          Washington|         USA|   0|
|   Teet|        Lva Tolstogo|              Kaluga|         RUS|   0|
| Mollom|Berchem-sainte-ag...|Brussels Hoofdste...|         BEL|   0|
|  63336|              London|             England|         GBR|   0|
|  TipiT|               Delft|        Zuid-Holland|         NLD|   0|
|  beweb|            Auckland|            Auckland|         NZL|   0|
|    CSC|        Falls Church|            Virginia|         USA|   0|
|    CBC|              Ottawa|             Ontario|         CAN|   0|
|  Orbro|           